In [1]:
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog

from sfincs_gfm.methods import gfm, breach_dikes, set_forcing

logger = setuplog(level="INFO")

INFO - log - hydroflows version: 0.1.0


In [ ]:
config = WorkflowConfig(
    region=Path("data", "region.geojson"),
    start_date="2022-10-11",
    end_date="2022-10-25",
    sf_root=Path("models/20m_final")
)

In [ ]:
wf = Workflow(name="baltic_gfm", config=config, root=Path().resolve())

In [ ]:
run_gfm = gfm.GFM(
    region=wf.config.region,
    starttime=wf.config.start_date,
    endtime=wf.config.end_date,
    output_dir=Path("gfm"),
)

wf.create_rule(run_gfm, rule_id="run_gfm")

In [ ]:
set_forcing = set_forcing.SetForcing(
    sfincs_inp=wf.config.sf_root/"sfincs.inp",
    output_dir="models/runs/test_run",
    start_time=wf.config.start_date,
    end_time=wf.config.end_time,
    wl_file=Path("data\waterlevel\pegelonline-barth-W-20231015-20231023.csv"),
    wind_file="https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/wind/historical/stundenwerte_FF_00298_19810101_20231231_hist.zip"
)

wf.create_rule(set_forcing, rule_id="set_forcing")

In [ ]:
breach_dikes = breach_dikes.BreachDikes(
    sfincs_inp=set_forcing.output.sfincs_out_inp,
    dike_locs=Path("data/dikes/all_dikes.gpkg"),
    breach_locs=Path("data/dikes/test_breaches.geojson"),
    output_dir="models/runs/test_run_breach"
)

wf.create_rule(breach_dikes, rule_id="breach_dikes")

In [ ]:
wf.plot_rulegraph(plot_rule_attrs=True)

In [ ]:
wf.dryrun()

In [ ]:
wf.run()